In [9]:
import pickle
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, StackingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from catboost import CatBoostClassifier
from tensorflow.keras.models import Sequential, save_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

# ===========================
# 1. Подготовка данных
# ===========================
# Признаки
features = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 
            'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

cat_features = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'thal']
num_features = list(set(features) - set(cat_features))

# Загрузка данных
data = pd.read_csv("heart_main.csv")
X = data[features]
y = data["num"]

# ===========================
# 2. Препроцессор
# ===========================
preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), num_features),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='most_frequent')),
            ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
        ]), cat_features)],
    remainder='passthrough')

# ===========================
# 3. Модели
# ===========================

# 1. KNN
knn_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', KNeighborsClassifier(
        metric='euclidean', n_neighbors=3, 
        weights='distance', algorithm='kd_tree', 
        leaf_size=10))
])
knn_pipe.fit(X, y)
with open('model_knn.pkl', 'wb') as f:
    pickle.dump(knn_pipe, f)

# 2. Gradient Boosting
gb_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', GradientBoostingClassifier(
        ccp_alpha=0.01471511413755825,
        learning_rate=0.5339651291062001,
        max_depth=1,
        max_leaf_nodes=15,
        min_samples_leaf=12,
        min_samples_split=20,
        n_estimators=99,
        random_state=42))
])
gb_pipe.fit(X, y)
with open('model_gb.pkl', 'wb') as f:
    pickle.dump(gb_pipe, f)

# 3. CatBoost
catboost_model = CatBoostClassifier(
    depth=9,
    grow_policy='Depthwise',
    iterations=57,
    learning_rate=0.03612670446188109,
    cat_features=[features.index(col) for col in cat_features],
    silent=True,
    random_seed=42
)
X_catboost = X.copy()
for col in cat_features:
    X_catboost[col] = X_catboost[col].astype(str)

catboost_model.fit(X_catboost, y)
with open('model_catboost.pkl', 'wb') as f:
    pickle.dump(catboost_model, f)

# 4. Random Forest
rf_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        max_features=0.7043559771674565,
        max_samples=0.9705153393013579,
        n_estimators=92,
        random_state=42))
])
rf_pipe.fit(X, y)
with open('model_rf.pkl', 'wb') as f:
    pickle.dump(rf_pipe, f)

# 5. Stacking
stacking_pipe = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', StackingClassifier(
        estimators=[
            ('dt', DecisionTreeClassifier(max_depth=5, random_state=42)),
            ('svc', SVC(kernel='poly', C=1, coef0=0.5, degree=3, gamma=0.01, probability=True, random_state=42)),
            ('knn', KNeighborsClassifier(leaf_size=10, metric='manhattan', n_neighbors=3, weights='distance')),
            ('gnb', GaussianNB(var_smoothing=1e-15))
        ],
        stack_method='auto',
        n_jobs=-1))
])
stacking_pipe.fit(X, y)
with open('model_stacking.pkl', 'wb') as f:
    pickle.dump(stacking_pipe, f)

# ===========================
# 4. Нейронная сеть
# ===========================
def create_nn(input_dim=13):
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Преобразуем данные для нейросети
preprocessed_X = preprocessor.fit_transform(X)

# Сохраняем препроцессор
with open('nn_preprocessor.pkl', 'wb') as f:
    pickle.dump(preprocessor, f)

# Обучаем и сохраняем нейросеть
nn_model = create_nn(input_dim=preprocessed_X.shape[1])
nn_model.fit(preprocessed_X, y, epochs=50, batch_size=16, verbose=0)
save_model(nn_model, 'model_nn.h5')

# ===========================
# 5. Успех
# ===========================
print("✅ Все модели обучены и сохранены!")
print("📦 Для нейросети сохранены:")
print("- Препроцессор: nn_preprocessor.pkl")
print("- Модель: model_nn.h5")


/Users/macbook/Library/Python/3.9/lib/python/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Все модели обучены и сохранены!
📦 Для нейросети сохранены:
- Препроцессор: nn_preprocessor.pkl
- Модель: model_nn.h5
